<a href="https://colab.research.google.com/github/ayaanzhaque/rae/blob/main/notebooks/xgboost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import relevant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
import warnings 

warnings.filterwarnings("ignore")

import data needed for model

In [2]:
data = pd.read_csv("data/train.csv")

preprocess data

In [3]:
data.drop(columns=["Dates"], inplace=True)

In [4]:
data = pd.get_dummies(data, columns=["PdDistrict", "DayOfWeek"], drop_first=True)

In [5]:
data

,Category,Descript,Resolution,Address,X,Y,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,...,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
0,WARRANTS,WARRANT ARREST,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,ROBBERY ON THE STREET WITH A GUN,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
878045,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE,600 Block of EDNA ST,-122.447364,37.731948,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
878046,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
878047,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


In [9]:
def transform_feature(dataframe):
    hour=dataframe['Hour']
    date= dataframe['Date']
    month=dataframe['Month']
    longitude=dataframe["X"]
    lattitude=dataframe["Y"]
    block= dataframe['Block_Ind']
    street=dataframe['Month']
    
    #Get binary vector for these varaibles and create columns
    weekday = pd.get_dummies(dataframe['DayOfWeek'])
    district = pd.get_dummies(dataframe['PdDistrict'])
    return pd.concat([hour, date, month, weekday, district, block, street, lattitude, longitude], axis=1)
    
encoder = preprocessing.LabelEncoder()
category=train['Category']
X = transform_feature(train)
y = encoder.fit_transform(category)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
parameters = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
              "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
              "min_child_weight" : [ 1, 3, 5, 7 ],
              "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
              "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]}

In [11]:
parameters = {"learning_rate"    : 	[0.20],
              "max_depth"        : [5],
              "min_child_weight" : [3],
              "gamma"            : [0.2],
              "colsample_bytree" : [0.7]}

In [12]:
%%time
xgb = GridSearchCV(XGBRegressor(random_state=42, objective='reg:squarederror'), param_grid=parameters)            
xgb.fit(X_train, y_train)

CPU times: user 14.6 s, sys: 381 ms, total: 15 s
Wall time: 7.12 s


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=42,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
        

In [13]:
xgb.score(X_train, y_train)

0.7696901988477335

In [14]:
xgb.score(X_test, y_test)

0.7200678819028002

In [15]:
xgb.best_params_

{'colsample_bytree': 0.7,
 'gamma': 0.2,
 'learning_rate': 0.2,
 'max_depth': 5,
 'min_child_weight': 3}

In [16]:
import pickle

In [20]:
best_pars = xgb.best_params_
best_model = xgb.best_estimator_
pickle.dump(xgb.best_estimator_, open("xgb_best_model.h5", "wb"))